In [8]:
# phase 1 imports
import point
import region
import utils
import math
import matplotlib
import matplotlib.pyplot as plt
listOfTrajectories = utils.importTrajectories("Trajectories")

In [9]:
def listTraj(listOfTrajectories):
    trajectories = []
    for traj in listOfTrajectories:
        trajectories += [[traj]]
    return trajectories

In [10]:
class Point:
    def __init__(self, x, y):
        self.X = x
        self.Y = y

def calculateDistance(point: Point, p1: Point, p2: Point):
    m = (p2.Y - p1.Y) / (p2.X - p1.X) if (p2.X - p1.X) != 0 else math.inf
    a = m
    b = -1
    c = -(m * p1.X - p1.Y)
    d = abs((a * point.X + b * point.Y + c)) / (math.sqrt(a * a + b * b))
    return d

def douglas_peucker(points, epsilon):
    dmax = 0
    index = 0
    end = len(points)
    for i in range(1, end - 1):
        d = calculateDistance(points[i], points[0], points[end - 1])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        rec_results1 = douglas_peucker(points[:index + 1], epsilon)
        rec_results2 = douglas_peucker(points[index:end], epsilon)

        return rec_results1[:-1] + rec_results2

    return [points[0], points[end - 1]]

def sliding_window_algorithm(points, epsilon, window_size):
    # Initialize an empty list to store the approximate trajectory points
    approximate_trajectory = []
    window = [points[0]]  # Start with the first point as the anchor point
    for i in range(1, len(points)):
        # Add the current point to the sliding window
        window.append(points[i])

        # Apply Douglas-Peucker algorithm to the current window points
        simplified_points = douglas_peucker(window, epsilon)

        # If the window size is exceeded, remove the last point from the window
        # (which caused the highest error) and start a new window with the last point as the anchor
        if len(simplified_points) > window_size:
            window.pop()
            approximate_trajectory.extend(simplified_points[:-1])  # Add the simplified points to the approximate trajectory
            window = [window[-1]]  # Start a new window with the last point as the anchor

    # After processing all points, add the last window points to the approximate trajectory
    approximate_trajectory.extend(window)
    
    return approximate_trajectory

def simplify_trajectories(trajectories, epsilon):
    simplified_trajectories = []
    for trajectory in trajectories:
        simplified_points = douglas_peucker(trajectory[0].points, epsilon)
        simplified_trajectories.append([simplified_points,trajectory[0].number])
    return simplified_trajectories

def vis(trajectories):
    plt.figure(figsize=(10,8))
    for trajec in trajectories:
        x = []
        y = []
        for traj in trajec[0]:
            x.append(traj.X)
            y.append(traj.Y)
        plt.plot(x, y, label="Trajectory {}".format(str(trajec[1])))
     
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Trajectory Visualization')
    plt.legend()
    plt.show() 




In [18]:
 
trajectories = listTraj(listOfTrajectories)
epsilon = 0.0001
simplified_trajectories = simplify_trajectories(trajectories, epsilon)    
# vis(simplified_trajectories)

# Set the threshold epsilon value and the sliding window size for the sliding window algorithm
epsilon_sliding = 0.0001
window_size = 3

# Apply the sliding window algorithm to each simplified trajectory
sliding_window_trajectories = []
for simplified_trajectory in simplified_trajectories:
    # Extract the points from the simplified trajectory (without the trajectory number)
    points_only = simplified_trajectory[0]
    
    # Apply the sliding window algorithm to the points
    sliding_window_points = sliding_window_algorithm(points_only, epsilon_sliding, window_size)

    # Add the resulting approximate trajectory (with trajectory number) to the list
    sliding_window_trajectories.append([sliding_window_points, simplified_trajectory[1]])
    
# # Visualize the sliding window trajectories
# vis(sliding_window_trajectories)
for i in sliding_window_trajectories[0][0]:
    print(i)

(0.0014788576577,0.0037183030576,2000-01-01:01:09:13)
(0.0016965927928,0.0037230323741,2000-01-01:01:09:25)
(0.0015009711712,0.0037330431655,2000-01-01:01:09:31)
(0.0015009711712,0.0037330431655,2000-01-01:01:09:31)
(0.0019141774775,0.002826573741,2000-01-01:01:10:30)
